---

# Generar Variables Climatológicas

Notebook para generar las variables climatológicas.

**Dependencias:**
- [jupyter](https://jupyter.org/)
- [matplotlib](https://matplotlib.org/)
- [numpy](https://numpy.org/)
- [pandas](https://pandas.pydata.org/)
- [scikit-learn](https://scikit-learn.org/stable/)
- [seaborn](https://seaborn.pydata.org/)
- [xgboost](https://xgboost.readthedocs.io/en/stable/)

**Fuentes de Datos:**
- [Humedad](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Humedad-del-Aire-2-metros/uext-mhny)
- [Precipitación](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Precipitaci-n/s54a-sgyg)
- [Presión](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Presi-n-Atmosf-rica/62tk-nxj5)
- [Temperatura](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Datos-Hidrometeorol-gicos-Crudos-Red-de-Estaciones/sbwg-7ju4)

In [2]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import xgboost as xgb
import warnings

from sklearn.metrics import mean_squared_error

In [3]:
plt.rcParams['figure.figsize'] = (12,8)
warnings.filterwarnings('ignore')

---

## Constantes

### Directorio de Datos

Se puede modificar la variable `DATA_DIR_NAME` por el nombre deseado para el directorio de los datos.

In [4]:
DATA_DIR_NAME = 'data'
PROJECT_DIR = os.path.abspath(os.pardir)
DATA_DIR = os.path.join(PROJECT_DIR, DATA_DIR_NAME)

### Lista de Departamentos

In [5]:
DEPARTS = [
    'ANTIOQUIA', 
    'ARAUCA', 
    'ATLANTICO', 
    'BOGOTA', 
    'BOLIVAR', 
    'BOYACA', 
    'CALDAS', 
    'CAQUETA', 
    'CASANARE', 
    'CAUCA', 
    'CESAR', 
    'CHOCO', 
    'CORDOBA', 
    'CUNDINAMARCA', 
    'GUAVIARE', 
    'HUILA', 
    'LA GUAJIRA', 
    'MAGDALENA', 
    'META', 
    'NARINO', 
    'NORTE DE SANTANDER', 
    'PUTUMAYO', 
    'QUINDIO', 
    'RISARALDA', 
    'SANTANDER', 
    'SUCRE', 
    'TOLIMA', 
    'VALLE DEL CAUCA'
]

### Mapeo Departamento - Región

| REGION       | DEPARTAMENTO            | | | REGION       | DEPARTAMENTO                 |
|:-------------|:------------------------|-|-|:-------------|:-----------------------------|
| Antioquia    | ANTIOQUIA               | | | Drummond     | CESAR                        |
| Arauca       | ARAUCA                  | | | Emec         | NARINO                       |
| Atlantico    | ATLANTICO               | | | GCM          | LA GUAJIRA, CESAR, MAGDALENA |
| BajoPutumayo | PUTUMAYO                | | | Guaviare     | GUAVIARE                     |
| Bolivar      | BOLIVAR                 | | | Huila        | HUILA                        |
| Boyaca       | BOYACA                  | | | Intercor     | LA GUAJIRA                   |
| Caldas       | CALDAS                  | | | Meta         | META                         |
| Cali         | VALLE DEL CAUCA         | | | Narino       | NARINO                       |
| Caqueta      | CAQUETA                 | | | NorSantander | NORTE DE SANTANDER           |
| Cartago      | VALLE DEL CAUCA         | | | Oxy          | CAQUETA                      |
| Casanare     | CASANARE                | | | Pereira      | RISARALDA                    |
| Cauca        | CAUCA                   | | | Planeta      | CORDOBA                      |
| Celsia       | VALLE DEL CAUCA, TOLIMA | | | Putumayo     | PUTUMAYO                     |
| Cerromatoso  | CORDOBA                 | | | Quindio      | QUINDIO                      |
| Choco        | CHOCO                   | | | Rubiales     | META                         |
| CiraInfanta  | SANTANDER               | | | Santander    | SANTANDER                    |
| Codensa      | BOGOTA                  | | | Tolima       | TOLIMA                       |
| CordobaSucre | CORDOBA, SUCRE          | | | TubosCaribe  | BOLIVAR                      |
| Cundinamarca | CUNDINAMARCA            | | | Tulua        | VALLE DEL CAUCA              |

In [6]:
DEPARTS_REGIONS = {
    'Antioquia': ['ANTIOQUIA'],
    'Arauca': ['ARAUCA'],
    'Atlantico': ['ATLANTICO'],
    'BajoPutumayo': ['PUTUMAYO'],
    'Bolivar': ['BOLIVAR'],
    'Boyaca': ['BOYACA'],
    'Caldas': ['CALDAS'],
    'Cali': ['VALLE DEL CAUCA'],
    'Caqueta': ['CAQUETA'],
    'Cartago': ['VALLE DEL CAUCA'],
    'Casanare': ['CASANARE'],
    'Cauca': ['CAUCA'],
    'Celsia': ['VALLE DEL CAUCA', 'TOLIMA'],
    'Cerromatoso': ['CORDOBA'],
    'Choco': ['CHOCO'],
    'CiraInfanta': ['SANTANDER'],
    'Codensa': ['BOGOTA'],
    'CordobaSucre': ['CORDOBA', 'SUCRE'],
    'Cundinamarca': ['CUNDINAMARCA'],
    'Drummond': ['CESAR'],
    'Emec': ['NARINO'],
    'GCM': ['LA GUAJIRA', 'CESAR', 'MAGDALENA'],
    'Guaviare': ['GUAVIARE'],
    'Huila': ['HUILA'],
    'Intercor': ['LA GUAJIRA'],
    'Meta': ['META'],
    'Narino': ['NARINO'],
    'NorSantander': ['NORTE DE SANTANDER'],
    'Oxy': ['CAQUETA'],
    'Pereira': ['RISARALDA'],
    'Planeta': ['CORDOBA'],
    'Putumayo': ['PUTUMAYO'],
    'Quindio': ['QUINDIO'],
    'Rubiales': ['META'],
    'Santander': ['SANTANDER'],
    'Tolima': ['TOLIMA'],
    'TubosCaribe': ['BOLIVAR'],
    'Tulua': ['VALLE DEL CAUCA'],
}

### Lista de Regiones

In [7]:
REGIONS = list(DEPARTS_REGIONS.keys())

### Rango de Fechas para Predicción
- `TRAIN_DATE` $\longrightarrow$ Fecha Inicial para Entrenamiento
- `START_DATE` $\longrightarrow$ Fecha Inicial para Predicción
- `END_DATE` $\longrightarrow$ Fecha Final

In [8]:
TRAIN_DATE = '2017-01-01'
START_DATE = '2020-10-01'
END_DATE = '2022-03-31'

---

## Análisis de Correlación

Se usaron las variables climatológicas y la demanda del _**2017-01-01**_ al _**2020-09-30**_:
- `PRESION`
- `HUMEDAD`
- `PRECIPITACION`
- `TEMPERATURA_HUMEDA`
- `TEMPERATURA_SECA`
- `HUMEDAD_RELATIVA_CALCULADA`
- `SENSACION_TERMICA_CALIENTE`
- `SENSACION_TERMICA_IDEAM`
- `DEMANDA`

Y se realizó una análisis de correlación usando el siguiente fragmento de código:

```py
columns = ['PRESION', 'HUMEDAD', 'PRECIPITACION', 'TEMPERATURA_HUMEDA', 'TEMPERATURA_SECA', 
           'HUMEDAD_RELATIVA_CALCULADA', 'SENSACION_TERMICA_CALIENTE', 'SENSACION_TERMICA_IDEAM', 'DEMANDA']

df = pd.read_csv(os.path.join(DATA_DIR, 'others', 'complete_dataset_type_2.csv'), usecols=columns)
                 
_ = plt.figure()
_ = sns.heatmap(df.corr(), annot=True, fmt='.4f', cmap='Greens', xticklabels=columns, yticklabels=columns)
_ = plt.title('Análisis de Correlación', size=18)
```

La ejecución de este código da como resultado el siguiente gráfico:

![Análisis de Correlación](../images/correlation_analysis.png)

Al analizar el gráfico, podemos notar que la `TEMPERATURA_HUMEDA` y la `TEMPERATURA_SECA` presentan un índice de correlación del 86.52%, por lo tanto se tomó la decisión de eliminar la variable `TEMPERATURA_HUMEDA` y las variables calculadas gracias a esta, i.e., la `HUMEDAD_RELATIVA_CALCULADA` y la `SENSACION_TERMICA_CALIENTE`.

Finalmente, se decidió renombrar las siguientes variables:
- `TEMPERATURA_SECA` $\longrightarrow$ `TEMPERATURA`
- `SENSACION_TERMICA_IDEAM` $\longrightarrow$ `SENSACION_TERMICA`

Por lo tanto, el conjunto de variables climatológicas queda conformado por:
- `PRESION`
- `HUMEDAD`
- `PRECIPITACION`
- `TEMPERATURA`
- `SENSACION_TERMICA`

---

## Creación del Árbol de Directorios

- \< *root_dir* \>
  - buffer
    - *humidity_filtered.csv*
    - *precipitation_filtered.csv*
    - *pressure_filtered.csv*
    - *temperature_filtered.csv*
  - input
    - humidity
      - \<*name*\>.csv
    - precipitation
      - \<*name*\>.csv
    - pressure
      - \<*name*\>.csv
    - temperature
      - \<*name*\>.csv
  - output
    - **dataset_clima.csv**

In [9]:
''' Generar Arbol de Directorios
Genera el árbol de directorios en los que se alojaran los datos necesarios para el correcto funcionamiento 
del script, así como los directorios para los datos filtrados y los datos de salida.

@param root_dir: Directorio raiz para el arbol de directorios
@param buffer_: Flag para la creación del directorio de los datos filtrados (Valor default = True)
@param input_: Flag para la creación del directorio de los datos de entrada (Valor default = True)
@param output_: Flag para la creación del directorio de los datos de salida (Valor default = True) 
'''
def generate_directory_tree(root_dir, buffer_=True, input_=True, output_=True):
    if buffer_:
        os.makedirs(os.path.join(root_dir, 'buffer'), exist_ok=True)
    
    if input_:
        os.makedirs(os.path.join(root_dir, 'input', 'humidity'), exist_ok=True)
        os.makedirs(os.path.join(root_dir, 'input', 'precipitation'), exist_ok=True)
        os.makedirs(os.path.join(root_dir, 'input', 'pressure'), exist_ok=True)
        os.makedirs(os.path.join(root_dir, 'input', 'temperature'), exist_ok=True)
    
    if output_:
        os.makedirs(os.path.join(root_dir, 'output'), exist_ok=True)

In [10]:
generate_directory_tree(DATA_DIR)

---

## Funciones

**Funciones Varias:**
- `verify_file_extension`

**Funciones para Dataframes:**
- `load_data`
- `filter_dataframe`
- `process_dataframes`
- `merge_dataframes`

**Funciones de Ajuste:**
- `fix_values`
- `fix_departs`
- `fix_columns`

**Funciones para Obtener Sensación Térmica:**
- `get_thermal_sensation`

**Funciones de Regresión XGBoost:**
- `evaluate`
- `xgboost_regression`

**Funciones para la Generación del Dataset Final:**
- `generate_final_dataset`

### Funciones Varias

In [11]:
''' Verificar Extensión
Verifica la extensión de un archivo deseado.

@param file_name: Nombre del archivo a verificar
@param target: Extensión que se desea verificar
@return: True si el archivo tiene la extensión deseada
@return: False si el archivo no tiene la extensión deseada
'''
def verify_file_extension(file_name, target):
    path, extension = os.path.splitext(file_name)
    extension = extension.lower()
    return extension == target

### Funciones para Dataframes

In [12]:
''' Cargar Datos
Realizar el cargue de los datos de una variable.

@param df_variable: Dataframe de los datos
@param variable_name: Nombre de la variable climatológica
@return: Dataframe de la variable correspondiente
'''
def load_data(df_variable, variable_name):
    for file_ in os.listdir(os.path.join(DATA_DIR, 'input', variable_name)):
        if verify_file_extension(file_, '.csv'):
            if df_variable is None:
                df_variable = pd.read_csv(os.path.join(DATA_DIR, 'input', variable_name, file_), sep=',', 
                                       usecols=['FechaObservacion', 'ValorObservado', 'Departamento'])
            else:
                df_variable = pd.concat([df_variable, 
                                         pd.read_csv(os.path.join(DATA_DIR, 'input', variable_name, file_), 
                                                     sep=',', 
                                                     usecols=['FechaObservacion', 
                                                              'ValorObservado', 
                                                              'Departamento'])])
    return df_variable


''' Filtrar Dataframe
Generar el dataset filtrado de una variable entre TRAIN_DATE y END_DATE.

@param df: Dataframe de los datos
@param variable_name: Nombre de la variable climatológica
@param date_format: Formato de la fecha del dataframe (Valor default = '%m/%d/%Y %I:%M:%S %p')
@param mode: Modo de filtrado (Default value = 'mean')
@return: Dataframe filtrado de la variable correspondiente
'''
def filter_dataframe(df, variable_name, date_format='%m/%d/%Y %I:%M:%S %p', mode='mean'):
    train = datetime.datetime.strptime(TRAIN_DATE, '%Y-%m-%d').date()
    end = datetime.datetime.strptime(END_DATE, '%Y-%m-%d').date()
    
    df = fix_departs(df, 'Departamento')
    df = df[df['Departamento'].isin(DEPARTS)]
    
    df['FechaObservacion'] = pd.to_datetime(df['FechaObservacion'], format=date_format)
    
    df['Fecha'] = df['FechaObservacion'].dt.date
    df['Hora'] = df['FechaObservacion'].dt.hour + 1
    
    df = df[(df['Fecha'] >= train) & (df['Fecha'] <= end)]
    
    df.drop(columns=['FechaObservacion'], inplace=True)
    
    if mode == 'mean':
        df = df.groupby(['Departamento', 'Fecha', 'Hora']).mean()
    elif mode == 'sum':
        df = df.groupby(['Departamento', 'Fecha', 'Hora']).sum()
    
    df.reset_index(inplace=True)
    
    df.to_csv(os.path.join(DATA_DIR, 'buffer', f'{variable_name}_filtered.csv'), index=False, header=True)
    print(f'- {variable_name}_filtered.csv guardado en buffer')
    
    return df
    

''' Procesar Dataframes
Procesar los dataframes de entrada para las variables climatológicas (Humedad, Precipitación, 
Presión y Temperatura) y generar los datasets filtrados entre TRAIN_DATE y END_DATE.

@param from_buffer: Flag para usar los datos filtrados previamente guardados en buffer (Valor default = False)
@return: Diccionario con los dataframes filtrados de las variables
'''
def process_dataframes(from_buffer=False):
    if from_buffer:
        humidity = pd.read_csv(os.path.join(DATA_DIR, 'buffer', 'humidity_filtered.csv'))
        precipitation = pd.read_csv(os.path.join(DATA_DIR, 'buffer', 'precipitation_filtered.csv'))
        pressure = pd.read_csv(os.path.join(DATA_DIR, 'buffer', 'pressure_filtered.csv'))
        temperature = pd.read_csv(os.path.join(DATA_DIR, 'buffer', 'temperature_filtered.csv'))

        humidity['Fecha'] = pd.to_datetime(humidity['Fecha'], infer_datetime_format=True)
        precipitation['Fecha'] = pd.to_datetime(precipitation['Fecha'], infer_datetime_format=True)
        pressure['Fecha'] = pd.to_datetime(pressure['Fecha'], infer_datetime_format=True)
        temperature['Fecha'] = pd.to_datetime(temperature['Fecha'], infer_datetime_format=True)
    else:
        humidity, precipitation, pressure, temperature = None, None, None, None
        
        print('Filtrando Dataset de Humedad...')
        humidity = load_data(humidity, 'humidity')
        humidity = filter_dataframe(humidity, 'humidity')
        print('- Completado\n')
        
        print('Filtrando Dataset de Precipitacion...')
        precipitation = load_data(precipitation, 'precipitation')
        precipitation = filter_dataframe(precipitation, 'precipitation', mode='sum')
        print('- Completado\n')
        
        print('Filtrando Dataset de Presion...')
        pressure = load_data(pressure, 'pressure')
        pressure = filter_dataframe(pressure, 'pressure')
        print('- Completado\n')
        
        print('Filtrando Dataset de Temperatura...')
        temperature = load_data(temperature, 'temperature')
        temperature = filter_dataframe(temperature, 'temperature')
        print('- Completado\n')
    
    variables = {
        'Humedad': humidity,
        'Precipitacion': precipitation,
        'Presion': pressure,
        'Temperatura': temperature
    }
    
    return variables


''' Merge Dataframes
Realiza el merge de 2 dataframes.

@param df_1: Primer dataframe a hacer merge
@param df_2: Segundo dataframe a hacer merge
@param on: Lista de columnas en las que hacer el merge
@param how: Modo para realizar el merge de los dataframes (Valor default = 'outer')
@return: Merged dataframe
'''
def merge_dataframes(df_1, df_2, on, how='outer'):
    df = pd.merge(df_1, df_2, how=how, on=on)
    return df

### Funciones de Ajuste

- **Ajuste Final de Columnas:**

| ANTES                      | DESPUES                    |
|:---------------------------|:---------------------------|
| Fecha                      | FECHA                      |
| Hora                       | HORA                       |
| Region                     | REGION                     |
| Presion                    | PRESION                    |
| Humedad                    | HUMEDAD                    |
| Precipitacion              | PRECIPITACION              |
| Temperatura                | TEMPERATURA                |
| Sensacion Termica          | SENSACION_TERMICA          |

In [13]:
''' Ajustar Valores
Ajustar los valores de una columna en un dataframe.

@param df: Dataframe de los datos
@param column: Nombre de la columna a la que se le ajustaran los valores
@param le_value: Valor mínimo para realizar el ajuste (Valor default = None)
@param ge_value: Valor máximo para realizar el ajuste (Valor default = None)
@return: Dataframe con los valores ajustados
'''
def fix_values(df, column, le_value=None, ge_value=None):
    if le_value is not None:
        df.loc[df[column] < le_value, column] = le_value
        
    if ge_value is not None:
        df.loc[df[column] > ge_value, column] = ge_value

    return df


''' Ajustar Departamentos
Ajustar (eliminar caracteres especiales) los departamentos de una columna en un dataframe.

@param df: Dataframe de los datos
@param column: Nombre de la columna a la que se le ajustaran los departamentos
@return: Dataframe con los departamentos ajustados
'''
def fix_departs(df, column):
    rename_departs = {
        'ATLÁNTICO': 'ATLANTICO',
        'BOGOTA D.C.': 'BOGOTA',
        'BOLÍVAR': 'BOLIVAR',
        'BOYACÁ': 'BOYACA',
        'CHOCÓ': 'CHOCO',
        'CÓRDOBA': 'CORDOBA',
        'NARIÑO': 'NARINO',
        'QUINDÍO': 'QUINDIO'
    }
    
    df[column].replace(to_replace=rename_departs, inplace=True)
    
    return df


''' Ajustar Columnas
Ajustar los nombres y el orden de las columnas en un dataframe.

@param df: Dataframe de los datos
@return: Dataframe con las columnas ajustadas
'''
def fix_columns(df):
    rename_columns = {
        'Fecha': 'FECHA',
        'Hora': 'HORA',
        'Region': 'REGION',
        'Presion': 'PRESION',
        'Humedad': 'HUMEDAD',
        'Precipitacion': 'PRECIPITACION',
        'Temperatura': 'TEMPERATURA',
        'Sensacion Termica': 'SENSACION_TERMICA'
    }
    
    df.rename(columns=rename_columns, inplace=True)
    
    column_names = [
        'FECHA',
        'HORA',
        'REGION',
        'PRESION',
        'HUMEDAD',
        'PRECIPITACION',
        'TEMPERATURA',
        'SENSACION_TERMICA'
    ]
    
    df = df.reindex(columns=column_names)
    
    return df

### Funciones para Obtener Sensación Térmica

**Variables:**

- Temperatura $(T)$
- Humedad $(H)$
- Sensación Térmica $(ST):$

$$ ST = -8.78469476 + 1.61139411 * T + 2.338548839 * H - 0.14611605 * T * H - 0.012308094 * T^2 - 0.016424828 * H^2 + 0.002211732 * T^2 * H + 0.00072546 * T * H^2 - 0.000003582 * T^2 * H^2 $$

In [14]:
''' Obtener Sensación Térmica
Generar la sensación térmica usando la formula descrita.

@param df: Dataframe de los datos
@param temperature_column: Columna de los valores de la temperatura
@param humidity_column: Columna de los valores de la humedad
@return: Dataframe con los valores de la sensación térmica
'''
def get_thermal_sensation(df, temperature_column, humidity_column):
    
    humidity = list(df[humidity_column])    
    temperature = list(df[temperature_column])
    
    thermal_sensation = list(map(lambda x, y: -8.78469476 + 1.61139411 * x + 2.338548839 * y - 0.14611605 * x * y - 0.012308094 * (x ** 2) - 0.016424828 * (y ** 2) + 0.002211732 * (x ** 2) * y + 0.00072546 * x * (y ** 2) - 0.000003582 * (x ** 2) * (y ** 2),
               temperature, humidity))
    
    df['Sensacion Termica'] = thermal_sensation
    
    return df

### Funciones de Regresión XGBoost

Se utiliza un predictor XGBoost para realizar la predicción de las variables climatológicas.

In [15]:
''' Evaluar modelo
Evalua el entrenamiento de un modelo usando las métricas R^2 y RMSE (Root Mean Square Error).

@param model: Modelo entrenado a evaluar
@param X_train: Conjunto X de datos de entrenamiento
@param y_train: Conjunto y de datos de entrenamiento
@param m_name: Nombre del modelo
@return: None
'''
def evaluate(model, X_train, y_train, m_name):
    y_pred_train = model.predict(X_train)
    
    r2_train = model.score(X_train, y_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    
    print(m_name)
    print('----------------------------------')
    print(f'Train R^2: {r2_train:.4f}')
    print(f'Train Root MSE: {rmse_train:.4f}\n')

    return None


''' Regresión XGBoost
Realiza la predicción de una variable usando un predictor XGBoost entre START_DATE y END_DATE.

@param df: Dataframe de los datos
@param date_column: Columna de los valores de la fecha
@param hour_column: Columna de los valores de la hora
@param values_column: Columna para los valores de la variable a predecir
@param variable_name: Nombre de la variable a predecir
@param region_name: Nombre de la región asociada a la predicción
@return: Dataframe con los valores de la sensación térmica
'''
def xgboost_regression(df, date_column, hour_column, values_column, variable_name, region_name):
    df['ordinal'] = df[date_column].apply(lambda x: x.toordinal())
    
    regres = df[['ordinal', hour_column, values_column]]
    regres.dropna(inplace=True)
    
    X = regres.iloc[:, :-1].values
    y = regres.iloc[:, -1].values
    
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree=0.4, learning_rate=0.1,
                                max_depth=50, alpha=1, n_estimators=250)
    
    # Fit to the training set
    xg_reg.fit(X, y)
    
    evaluate(xg_reg, X, y, f'XGBoost {region_name} {variable_name}')
    
    start = datetime.datetime.strptime(START_DATE, '%Y-%m-%d').date().toordinal()
    end = datetime.datetime.strptime(END_DATE, '%Y-%m-%d').date().toordinal()
    
    days = np.arange(start, end + 1) # Dias 
    hours = np.arange(24) + 1 # Horas
    
    no_days = len(days) # Cantidad de Dias
    nums = no_days * 24 # Numero de Registros
    
    matrix = np.array([np.repeat(days, 24), np.tile(hours, no_days)]).T
    
    predict = xg_reg.predict(matrix)
    
    df_predict = pd.DataFrame({'ordinal': matrix[:, 0], hour_column: matrix[:, 1], values_column: predict})
    df_final = pd.concat([regres, df_predict])
    df_final.drop_duplicates(subset=['ordinal', hour_column], inplace=True, keep='first')
    
    df_final = df_final.astype({'ordinal': 'int64', hour_column: 'int8'}, copy=True)
    df_final = df_final[(df_final['ordinal'] >= start) & (df_final['ordinal'] <= end)]
    
    df_final[date_column] = [datetime.date.fromordinal(i) for i in df_final['ordinal']]
    df_final[date_column] = pd.to_datetime(df_final[date_column], infer_datetime_format=True)
    
    df_final.rename(columns={values_column: variable_name}, inplace=True)
    df_final.drop(columns=['ordinal'], inplace=True)
    df_final.sort_values([date_column, hour_column], inplace=True)
    
    return df_final

### Funciones para la Generación del Dataset Final

In [16]:
''' Generar Dataset Final
Generar el dataset final de las variables climatológicas entre START_DATE y END_DATE.

@param variables: Datasets filtrados de las variables climatológicas por departamentos
@return: Dataframe final con para las variables climatológicas
'''
def generate_final_dataset(variables):
    print('Generando el dataset final...\n')
    
    df_final = None
    for reg, deps in DEPARTS_REGIONS.items():
        df_region = None
        for var, df_var in variables.items():
            df_train = df_var[df_var['Departamento'].isin(deps)]
            df_train.drop(columns=['Departamento'], inplace=True)
            df_train = df_train.groupby(['Fecha', 'Hora']).mean()
            df_train.reset_index(inplace=True)

            if df_region is None:
                df_region = xgboost_regression(df_train, 'Fecha', 'Hora', 'ValorObservado', var, reg)
            else:
                df_region = merge_dataframes(df_region, 
                                             xgboost_regression(df_train, 'Fecha', 'Hora', 'ValorObservado', 
                                                                var, reg), 
                                             ['Fecha', 'Hora'])

        df_region['Region'] = [reg] * len(df_region)

        if df_final is None:
            df_final = df_region
        else:
            df_final = pd.concat([df_final, df_region])
            
    df_final = fix_values(df_final, 'Humedad', le_value=0, ge_value=100)
    df_final = fix_values(df_final, 'Precipitacion', le_value=0)
    df_final = get_thermal_sensation(df_final, 'Temperatura', 'Humedad')
    df_final = fix_columns(df_final)
    
    df_final.to_csv(os.path.join(DATA_DIR, 'output', 'dataset_clima.csv'), index=False, header=True)
    
    print('- dataset_clima.csv guardado en output')
    print('- Completado')
    
    return df_final

---

## Ejecución del Script

### Procesar Dataframes

In [17]:
variables = process_dataframes()

Filtrando Dataset de Humedad...
- humidity_filtered.csv guardado en buffer
- Completado

Filtrando Dataset de Precipitacion...
- precipitation_filtered.csv guardado en buffer
- Completado

Filtrando Dataset de Presion...
- pressure_filtered.csv guardado en buffer
- Completado

Filtrando Dataset de Temperatura...
- temperature_filtered.csv guardado en buffer
- Completado



### Generar Dataset Final

In [18]:
df_final = generate_final_dataset(variables)

Generando el dataset final...

XGBoost Antioquia Humedad
----------------------------------
Train R^2: 0.8877
Train Root MSE: 4.5696

XGBoost Antioquia Precipitacion
----------------------------------
Train R^2: 0.4079
Train Root MSE: 13.0361

XGBoost Antioquia Presion
----------------------------------
Train R^2: 0.9164
Train Root MSE: 7.0386

XGBoost Antioquia Temperatura
----------------------------------
Train R^2: 0.9216
Train Root MSE: 0.9655

XGBoost Arauca Humedad
----------------------------------
Train R^2: 0.8895
Train Root MSE: 9.7375

XGBoost Arauca Precipitacion
----------------------------------
Train R^2: 0.2630
Train Root MSE: 3.5073

XGBoost Arauca Presion
----------------------------------
Train R^2: 0.9819
Train Root MSE: 20.4230

XGBoost Arauca Temperatura
----------------------------------
Train R^2: 0.9591
Train Root MSE: 1.9156

XGBoost Atlantico Humedad
----------------------------------
Train R^2: 0.8366
Train Root MSE: 5.2038

XGBoost Atlantico Precipitacion


XGBoost Drummond Presion
----------------------------------
Train R^2: 0.9181
Train Root MSE: 7.0290

XGBoost Drummond Temperatura
----------------------------------
Train R^2: 0.8373
Train Root MSE: 1.7993

XGBoost Emec Humedad
----------------------------------
Train R^2: 0.8362
Train Root MSE: 5.8300

XGBoost Emec Precipitacion
----------------------------------
Train R^2: 0.2665
Train Root MSE: 5.9721

XGBoost Emec Presion
----------------------------------
Train R^2: 0.8709
Train Root MSE: 11.3217

XGBoost Emec Temperatura
----------------------------------
Train R^2: 0.8789
Train Root MSE: 1.2817

XGBoost GCM Humedad
----------------------------------
Train R^2: 0.8299
Train Root MSE: 5.3718

XGBoost GCM Precipitacion
----------------------------------
Train R^2: 0.2949
Train Root MSE: 2.7285

XGBoost GCM Presion
----------------------------------
Train R^2: 0.9316
Train Root MSE: 3.2101

XGBoost GCM Temperatura
----------------------------------
Train R^2: 0.9159
Train Root MSE:

In [19]:
df_final

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA
0,2020-10-01,1,Antioquia,840.033333,88.215943,14.805,16.990926,18.713188
1,2020-10-01,2,Antioquia,842.750000,88.718752,26.300,17.038800,18.488753
2,2020-10-01,3,Antioquia,842.125000,90.841265,8.955,16.834266,17.644681
3,2020-10-01,4,Antioquia,839.300000,93.334961,20.504,16.460470,16.610112
4,2020-10-01,5,Antioquia,842.425000,93.536953,34.805,16.401083,16.532308
...,...,...,...,...,...,...,...,...
13123,2022-03-31,20,Tulua,832.350000,99.000000,2.300,17.800000,13.863864
13124,2022-03-31,21,Tulua,821.050000,99.000000,0.200,17.933333,13.915338
13125,2022-03-31,22,Tulua,821.800000,99.000000,0.000,17.866667,13.888839
13126,2022-03-31,23,Tulua,822.350000,98.000000,0.000,17.600000,14.285036


### Verificar Dataset Final

In [20]:
df_clima = pd.read_csv(os.path.join(DATA_DIR, 'output', 'dataset_clima.csv'))

In [21]:
df_clima

,FECHA,HORA,REGION,PRESION,HUMEDAD,PRECIPITACION,TEMPERATURA,SENSACION_TERMICA
0,2020-10-01,1,Antioquia,840.033333,88.215943,14.805,16.990926,18.713188
1,2020-10-01,2,Antioquia,842.750000,88.718752,26.300,17.038800,18.488753
2,2020-10-01,3,Antioquia,842.125000,90.841265,8.955,16.834266,17.644681
3,2020-10-01,4,Antioquia,839.300000,93.334961,20.504,16.460470,16.610112
4,2020-10-01,5,Antioquia,842.425000,93.536953,34.805,16.401083,16.532308
...,...,...,...,...,...,...,...,...
498859,2022-03-31,20,Tulua,832.350000,99.000000,2.300,17.800000,13.863864
498860,2022-03-31,21,Tulua,821.050000,99.000000,0.200,17.933333,13.915338
498861,2022-03-31,22,Tulua,821.800000,99.000000,0.000,17.866667,13.888839
498862,2022-03-31,23,Tulua,822.350000,98.000000,0.000,17.600000,14.285036
